In [ ]:
from typing import TypedDict,Annotated
from langgraph.graph import END, StateGraph, START,add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage,AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv

load_dotenv()

class ChildState(TypedDict):
    messages : Annotated[list,add_messages]

search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm_with_tools = llm.bind_tools(tools=tools)

def agent(state:ChildState):
    return {
        "messages":[llm_with_tools.invoke(state["messages"])],
    }

def tools_router(state:ChildState):
    last_message = state["messages"][-1]

    if(hasattr(last_message,"tool_calls") and len(last_message.tool_calls)>0):
        return "tool_node"
    else:
        return END
    
tool_node = ToolNode(tools=tools)

subgraph = StateGraph(ChildState)

subgraph.add_node("agent",agent)
subgraph.add_node("tool_node",tool_node)
subgraph.set_entry_point("agent")

subgraph.add_conditional_edges("agent",tools_router)
subgraph.add_edge("tool_node","agent")

search_app=subgraph.compile()





NameError: name 'add_messages' is not defined